In [13]:
import os
import sys

import glob
import json

# from tqdm.auto import tqdm
from tqdm import tqdm

import time

import msgpack
import numpy as np
import pandas as pd

In [3]:
sys.path.append(os.path.abspath('../kcg-ml-image-pipeline/'))
from utility.minio.cmd import connect_to_minio_client

In [4]:
ROOT = '../dataset/'

DATASETs = [
    'environmental', 
    'character', 
    'icons', 
    'mech', 
    'waifu',
    'propaganda-poster'
]

In [5]:
# MINIO_ADDRESS = "123.176.98.90:9000"
MINIO_ADDRESS = "192.168.3.5:9000"
access_key = "GXvqLWtthELCaROPITOG"
secret_key = "DmlKgey5u0DnMHP30Vg7rkLT0NNbNIGaM8IwPckD"
bucket_name = 'datasets'

In [6]:
client = connect_to_minio_client(MINIO_ADDRESS, access_key, secret_key)

Connecting to minio client...
Successfully connected to minio client...


# download rank data

In [7]:
def download_rank_data(dataset_name):

    objects = client.list_objects(bucket_name, f'{dataset_name}/data/ranking/aggregate/', recursive=False)
    objects = list(tqdm(objects, leave=False))

    for obj in tqdm(objects, leave=False):
        
        object_name = obj.object_name
        
        if not object_name.lower().endswith('.json'):
            continue
    
        out_path = os.path.join(ROOT, 'ranking', object_name.replace('/data/ranking/aggregate/', '/'))
        
        if os.path.exists(out_path):
            continue
            
        try:
            client.fget_object(bucket_name, object_name, out_path)
        except KeyboardInterrupt:
            break
        except:
            continue

In [8]:
for dataset_name in DATASETs:
    download_rank_data(dataset_name)

# from rank

In [7]:
def download_ranked_data(dataset_name):

    file_paths = set()
    
    for fname in os.listdir(os.path.join(ROOT, 'ranking', dataset_name)):
        
        info = json.load(open(os.path.join(ROOT, 'ranking', dataset_name, fname)))
        
        image_path_1 = info['image_1_metadata']['file_path']
        
        file_paths.add(image_path_1)
        
        image_path_2 = info['image_2_metadata']['file_path']
            
        file_paths.add(image_path_2)
    
    for file_name in tqdm(file_paths, leave=False):
            
        # output_path = file_name.replace('.jpg', '_clip.msgpack').replace(f'datasets/{dataset_name}/', os.path.join(ROOT, f'clip/{dataset_name}/'))
        # file_name = file_name.replace('.jpg', '_clip.msgpack').replace('datasets/', '')
            
        output_path = file_name.replace('.jpg', '_data.msgpack').replace(f'datasets/{dataset_name}/', os.path.join(ROOT, f'data/{dataset_name}/'))
        file_name = file_name.replace('.jpg', '_data.msgpack').replace('datasets/', '')
            
        # output_path = file_name.replace(f'datasets/{dataset_name}/', os.path.join(ROOT, f'image/{dataset_name}/'))
        # file_name = file_name.replace('datasets/', '')

        if os.path.exists(output_path):
            continue
        
        try:
            client.fget_object(bucket_name, file_name, output_path)
        except KeyboardInterrupt:
            break
        except:
            continue
        

In [8]:
for dataset_name in DATASETs:
    download_ranked_data(dataset_name)

# save json

In [9]:
def save_json(dataset_name):

    paths = sorted(glob.glob(os.path.join(ROOT, 'data', dataset_name, '**/*_data.msgpack')))
    
    job_uuids = list()
    file_paths = list()
    file_hashs = list()
    positive_prompts = list()
    negative_prompts = list()
    creation_times = list()
    for path in tqdm(paths, leave=False):
    
        if not os.path.exists(path.replace('_data.msgpack', '_clip.msgpack').replace('/data/', '/clip/')):
            continue
    
        with open(path, 'rb') as f:
            mp = msgpack.load(f)
        
        job_uuids.append(mp['job_uuid'])
        file_paths.append(mp['file_path'])
        file_hashs.append(mp['file_hash'])
        positive_prompts.append(mp['positive_prompt'])
        negative_prompts.append(mp['negative_prompt'])
        creation_times.append(mp['creation_time'])
    
    os.makedirs(os.path.join('data', dataset_name), exist_ok=True)

    df = pd.DataFrame(
        zip(file_paths, creation_times, job_uuids, positive_prompts, negative_prompts, file_hashs), 
        columns=['file_path', 'creation_time', 'job_uuid', 'positive_prompt', 'negative_prompt', 'file_hash']
    )

    df.drop_duplicates(['file_hash'], inplace=True)
    df.set_index('file_hash', inplace=True)
    
    json.dump(
        df.to_dict(orient='index'),
        open(os.path.join('data', dataset_name, 'data.json'), 'w')
    )

In [14]:
for dataset_name in DATASETs:
    save_json(dataset_name)

# download whole dataset

In [ ]:
for dataset_name in DATASETs:
    objects = list()
    for i in tqdm(client.list_objects(bucket_name, dataset_name, recursive=True)):
        objects.append(i)
    # objects = sorted(objects, key=lambda x: x.object_name)

In [35]:
len([i for i in objects if i.object_name.endswith('.jpg')])

123053

In [36]:
for obj in tqdm([i for i in objects if i.object_name.endswith('.jpg')][::10]):
# for obj in tqdm(objects):
    
    object_name = obj.object_name
    
    # if not object_name.lower().endswith((".jpg", ".png", ".jpeg")):
    #     continue
    # out_path = os.path.join(ROOT, 'image', object_name)
    
    # if not object_name.lower().endswith('_clip.msgpack'):
    #     continue
    # out_path = os.path.join(ROOT, 'clip', object_name)

    if object_name.lower().endswith((".jpg", ".png", ".jpeg")):
        out_path = os.path.join(ROOT, 'image', object_name)
    # if object_name.lower().endswith('_clip.msgpack'):
    #     out_path = os.path.join(ROOT, 'clip', object_name)
    # elif object_name.lower().endswith('_data.msgpack'):
    #     out_path = os.path.join(ROOT, 'data', object_name)
    # elif object_name.lower().endswith('.json') and '/ranking/aggregate/' in object_name.lower():
    #     out_path = os.path.join(ROOT, 'ranking', object_name.replace('/data/ranking/aggregate/', '/'))
    else:
        continue
    
    if os.path.exists(out_path):
        continue
    
    try:
        client.fget_object(bucket_name, object_name, out_path)
    except KeyboardInterrupt:
        break
    except:
        continue

  0%|          | 0/12306 [00:00<?, ?it/s]

In [37]:
image_paths = list()

for dname in os.listdir(os.path.join(ROOT, 'image', dataset_name)):
    for fname in os.listdir(os.path.join(ROOT, 'image', dataset_name, dname)):
        if not fname.endswith('.jpg'):
            continue
        image_paths.append(os.path.join(dataset_name, dname, fname))

# for dname in os.listdir(os.path.join(ROOT, 'clip', dataset_name)):
#     for fname in os.listdir(os.path.join(ROOT, 'clip', dataset_name, dname)):
#         if not fname.endswith('_clip.msgpack'):
#             continue
#         image_paths.append(os.path.join(dataset_name, dname, fname))

In [38]:
for path in tqdm(image_paths):
    
    # object_name = path.replace('_clip.msgpack', '_data.msgpack')
    
    object_name = path.replace('.jpg', '_data.msgpack')
    out_path = os.path.join(ROOT, 'data', object_name)
    
    # object_name = path.replace('.jpg', '_clip.msgpack')
    # out_path = os.path.join(ROOT, 'clip', object_name)
    
    if os.path.exists(out_path):
        continue
    
    try:
        client.fget_object(bucket_name, object_name, out_path)
    except KeyboardInterrupt:
        break
    except:
        continue
    

  0%|          | 0/8816 [00:00<?, ?it/s]

In [39]:
for path in tqdm(image_paths):
    
    # object_name = path.replace('_clip.msgpack', '_data.msgpack')
    
    # object_name = path.replace('.jpg', '_data.msgpack')
    # out_path = os.path.join(ROOT, 'data', object_name)
    
    object_name = path.replace('.jpg', '_clip.msgpack')
    out_path = os.path.join(ROOT, 'clip', object_name)
    
    if os.path.exists(out_path):
        continue
    
    try:
        client.fget_object(bucket_name, object_name, out_path)
    except KeyboardInterrupt:
        break
    except:
        continue
    

  0%|          | 0/8816 [00:00<?, ?it/s]

In [20]:
msp_paths = list()
vectors = list()

for dname in os.listdir(os.path.join(ROOT, 'clip', dataset_name)):
    for fname in os.listdir(os.path.join(ROOT, 'clip', dataset_name, dname)):
        
        msp_path = os.path.join(dataset_name, dname, fname)

        with open(os.path.join(ROOT, 'clip', msp_path), 'rb') as f:
            data = f.read()
        decoded_data = msgpack.unpackb(data)
        
        msp_paths.append(msp_path)
        vectors.append(np.array(decoded_data['clip-feature-vector']))

In [21]:
EMB_PATH = 'data/environmental/clip_vision_emb.npz'

In [22]:
np.savez(
    EMB_PATH,
    file_paths=np.array(msp_paths),
    image_embeds=np.concatenate(vectors, axis=0)
)